# Calculate ICE Demand (EVI-Pro Reverse Engineer)

## Imports

In [13]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from fiona.crs import from_epsg

import warnings
warnings.simplefilter("ignore")

# Custom utilities
from src.utils import Vehicle, Simulation, ChargingEvents, HexGrid
from src.models import Random_Sample_Charge_Location_Model, Linear_Kwh_Model

## Load Trajectories

In [14]:
data_all = pd.read_csv('../data/raw/part-00167-tid-8316490494947456795-71092cdb-87a8-47f5-9d32-296121158b22-3052-1.c000.csv', chunksize=5000)

In [15]:
for data in data_all:
    data = data[data.battery_pack.isna()]
    dataGeo = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.decr_lng, data.decr_lat), crs=from_epsg(4326))
    dataGeo['t'] = pd.to_datetime(dataGeo.element_time_local, utc=True)
    dataGeo = dataGeo.set_index('t')
    traj_collection = mpd.TrajectoryCollection(dataGeo, 'hashed_vin')
    break

## Create Vehicles

In [16]:
# For each vehicle trajectory create a vehicle object
vehicles = []
for traj in traj_collection.trajectories:
    vehicles.append(Vehicle(traj))

## Create Trained Models

In [17]:
# Train the linear model
linear_kwh_model = Linear_Kwh_Model()
linear_kwh_model.train()

In [18]:
# No training required for this model
random_sampling_model = Random_Sample_Charge_Location_Model()

## Run Simulation

In [19]:
sim = Simulation(vehicles[0:100], random_sampling_model, linear_kwh_model)

In [20]:
# See if any loops get caught!
sim.run()

## Compile Charging Events

In [21]:
# Takes a while
events_list = []

for v in vehicles:
    if v.charge_events:
        events_list.append(v.charge_events)

charge_events = ChargingEvents(events_list)

## Create Demand Curve and Geopandas Dataframe

In [22]:
hourly_charge_events_gdf = charge_events.to_hourly()
charge_events_gdf = charge_events.to_geopandas()

## Mapping

In [23]:
# Create a grid of resolution
grid = HexGrid(8)

# Join the dataframe with the grid
grid.sjoin(hourly_charge_events_gdf)

In [24]:
# Plot the energy values
hmap = grid.plot(value='energy')

TypeError: hexagons_dataframe_to_geojson() got multiple values for argument 'df_hex'

In [ ]:
hmap